## Arrays and summation

In [1]:
# Power generation cost minimization problem

# Power generation (Pg) and cost (Cg) arrays

# ID     |     Cost      |   Power generation limit
# =================================================
# 0      |     0.10      |   20 kW
# 1      |     0.05      |   10 kW
# 2      |     0.30      |   40 kW
# 3      |     0.40      |   50 kW
# 4      |     0.01      |    5 kW


# Load points (Pd)
# ID     |     Load demand (kW)
# ============================
# 0      |     50 kW
# 1      |     20 kW
# 2      |     30 kW

# * Only generators 0 and 3 are available for the load point 0

In [ ]:
# Equations

# min sum_i_g(Cg[i_g] * Pg[i_g])     # Minimize cost
# where i_g = 0, 1, 2, 3, 4      # Generator IDs

# sum_i_g(Pg[i_g]) = sum_i_c(Pd[i_d])   # Power balance

# P_d(0) <= P_g(0) + P_g(3)   # Load point 0
# P_g(i_g) >= 0 for i_g = 0, 1, 2, 3, 4   # Power generation limits (no negative power generation)
# P_g(i_g) <= P_g(i_g)^LIMIT for i_g = 0, 1, 2, 3, 4   # Power generation limits (no power generation above the limit)



In [5]:
import numpy as np

In [6]:
# put geenrator data into an array

gen_data = np.array([[0.10, 20], [0.05, 10], [0.30, 40], [0.40, 50], [0.01, 5]])

load_data = np.array([50, 20, 30])

In [9]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

# create a model
data_gen = pd.DataFrame(gen_data, columns=['cost', 'power_limit'])
data_load = pd.DataFrame(load_data, columns=['load_demand'])
number_of_generators = len(data_gen)

# check the data
print(data_gen)
print(data_load)

   cost  power_limit
0  0.10         20.0
1  0.05         10.0
2  0.30         40.0
3  0.40         50.0
4  0.01          5.0
   load_demand
0           50
1           20
2           30


In [32]:
# create a model

model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(number_of_generators),  # one dimension only
                   # range(len(data_load)),  # possible second dimension
                   bounds = (0, None))   # power generation

P_g = model.Pg

# print out the model
model.pprint()

1 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

2 Declarations: Pg_index Pg


In [33]:
[P_g[g] for g in data_gen.index]

In [34]:
# constraints

# power balance
pg_sum = sum([P_g[g] for g in data_gen.index])    # sum of all generators
model.balance = pyo.Constraint(expr = pg_sum == sum(data_load['load_demand']))    # power balance


# power generation limits
model.cond = pyo.Constraint(expr = P_g[0] + P_g[3] >= data_load['load_demand'][0])    # load point 0

model.limits = pyo.ConstraintList()   # create a constraint list

for g in data_gen.index:  # loop over all generators
    model.limits.add(expr = P_g[g] <= data_gen['power_limit'][g])   # add a constraint to the list

# print out the model
model.pprint()

2 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    limits_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

3 Constraint Declarations
    balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    cond : Size=1, I

In [35]:
# Objective function
# Here we use a list comprehension to create a list of expressions
# and then sum them up

cost_sum = sum([P_g[g] * data_gen['cost'][g] for g in data_gen.index])
model.obj = pyo.Objective(expr = cost_sum, sense=minimize)



In [38]:
# solve the model
opt = SolverFactory('gurobi')
results = opt.solve(model)

# print out the results
data_gen['Pg'] = [pyo.value(P_g[g]) for g in data_gen.index]
print(data_gen)

   cost  power_limit    Pg
0  0.10         20.0  20.0
1  0.05         10.0  10.0
2  0.30         40.0  35.0
3  0.40         50.0  30.0
4  0.01          5.0   5.0


In [39]:
# Notes on Pyomo: Double Summations and Variables with 2 or more dimensions

# Pyomo provides a flexible and powerful way to define variables and constraints for optimization problems.
# One key feature is the ability to define variables with multiple dimensions.
# This is useful for problems with multiple decision variables, such as the power generation problem above.
# Let's us x[i, j, k] to represent a variable with three dimensions.

# Declaring the variable
from pyomo.environ import *

# Declaring sets
set_I = [1, 2, 3, 4, 5]
set_J = [10, 20, 30, 40]
set_K = ["A", "B"]

# Declaring the x variable and creating the model (consdering bounds of 0 and 100
model = ConcreteModel()
model.x = Var(set_I, set_J, set_K, bounds=(0, 100))

In [41]:
# Creating a summation with three dimensions or indices

sum_x = sum(model.x[i, j, k] for i in set_I for j in set_J for k in set_K)

# This creates a new variable that is the sum of all the values of x[i, j, k] for all values of i, j, and k.
# you can use this variable in a constraint or objective function.



In [42]:
# Creating the constraint with the summation (where the sum of all x[i, j, k] is less than or equal to 100)

model.constraint = Constraint(expr = sum_x <= 100)

